In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pl
import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Conv2D, Flatten, MaxPooling2D, AveragePooling2D, LeakyReLU, concatenate
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow import clip_by_value
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
import pickle

In [ ]:
train_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\train_images.parquet').read().to_pandas()
test_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\test_images.parquet').read().to_pandas()
val_images = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Images\\val_images.parquet').read().to_pandas()

In [ ]:
train_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\train_labels.parquet').read().to_pandas()
test_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\test_labels.parquet').read().to_pandas()
val_labels = pq.ParquetDataset('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Training Data\\Labels\\val_labels.parquet').read().to_pandas()

In [ ]:
train_images = np.array(train_images).reshape(train_images.shape[0],48,48,1)
test_images = np.array(test_images).reshape(test_images.shape[0],48,48,1)
val_images = np.array(val_images).reshape(val_images.shape[0],48,48,1)

In [ ]:
cnn_max = Input(shape=(48,48,1))
x = Conv2D(100, kernel_size=6, activation=LeakyReLU(alpha=0.05))(cnn_max)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(220, kernel_size=2, activation=LeakyReLU(alpha=0.05))(x)
x = MaxPooling2D(pool_size=(2,2))(x)
cnn_max_out = Flatten()(x)

In [ ]:
cnn_avg = Input(shape=(48,48,1))
x = Conv2D(100, kernel_size=6, activation=LeakyReLU(alpha=0.05))(cnn_avg)
x = AveragePooling2D(pool_size=(2,2))(x)
x = Conv2D(220, kernel_size=2, activation=LeakyReLU(alpha=0.05))(x)
x = AveragePooling2D(pool_size=(2,2))(x)
cnn_avg_out = Flatten()(x)

In [ ]:
cnn_concat = concatenate([cnn_max_out, cnn_avg_out])
x = Dropout(0.5)(cnn_concat)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
cnn_concat_out = Dense(7, activation='softmax')(x)

In [ ]:
optimizer = optimizers.Adam(learning_rate=0.000075)

In [ ]:
cnn = Model(inputs=[cnn_max, cnn_avg], outputs=cnn_concat_out)

In [ ]:
cnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
cnn_history = cnn.fit([train_images, train_images], train_labels, validation_data=([test_images, test_images], test_labels),
                      batch_size=200, epochs=200)

In [ ]:
cnn.save('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Models\\Convolutional_Stacked\\Model.h5')
with open('C:\\Users\\rbush\\Documents\\Projects\\Computer Vision\\Facial Expressions\\Models\\Convolutional_Stacked\\Training History.pkl','wb') as outfile:
    pickle.dump(cnn_history.history, outfile)

In [ ]:
cnn_predicted_categories = np.argmax(cnn.predict([val_images, val_images]), axis=1)

In [ ]:
val_labels = np.argmax(np.array(val_labels), axis=1)

In [ ]:
cnn_cm = confusion_matrix(cnn_predicted_categories, val_labels, labels=[0,1,2,3,4,5,6])

In [ ]:
cnn_cm

In [ ]:
cnn_accuracy = (cnn_cm[0,0]+cnn_cm[1,1]+cnn_cm[2,2]+cnn_cm[3,3]+cnn_cm[4,4]+cnn_cm[5,5]+cnn_cm[6,6])/np.sum(cnn_cm)
print(f'Conv-Net Accuracy: {round(100*cnn_accuracy,1)}%')